In [4]:
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
import ccdproc as ccdp

import numpy as np
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture

In [5]:
light_file_collection = ccdp.ImageFileCollection('../reduced-lights_2022_03_20/Luminance')
data = [i for i in light_file_collection.data()]
headers = [i for i in light_file_collection.headers()]

In [6]:
import pandas as pd

def find_sources(array):
    mean, median, std = sigma_clipped_stats(array, sigma=2.5)
    daofind = DAOStarFinder(fwhm=9.0, threshold=5.*std)
    sources = daofind(array - median) # Needs to be changed to local bg
    return sources

def image_trimmer(sources, array):
    local_df = sources.to_pandas()
    image_size = array.shape
    x_trim = image_size[1] * 0.05
    y_trim = image_size[0] * 0.05
    local_df = local_df[(local_df['xcentroid'] - x_trim > 0) & (local_df['xcentroid'] + x_trim < image_size[1])]
    local_df = local_df[(local_df['ycentroid'] - y_trim > 0) & (local_df['ycentroid'] + y_trim < image_size[0])]
    return local_df.reset_index(drop = True).drop('id', axis = 'columns')

def excluding_duplicates(df, fwhm):
    #Taking initial comparison values from first row
    xcenter, ycenter, flux = df.iloc[0][['xcentroid', 'ycentroid', 'flux']]
    last_index = 0
    #Including first row in result
    filters = [True]

    #Skipping first row in comparisons
    for index, row in df.iloc[1:].iterrows():
        if (xcenter - 3*fwhm <= row['xcentroid'] <= 3*fwhm + xcenter) or (ycenter - 3*fwhm <= row[
            'ycentroid'] <= 3*fwhm + ycenter):
            # Once we have the two that are very close to each other we want to keep the one with the highest flux
            if df.iloc[last_index]['flux'] > row['flux']:
                filters.append(False)
            else:
                filters[last_index] = False
                filters.append(True)
                xcenter = row['xcentroid']
                ycenter = row['ycentroid']
                last_index = index
        else:
            filters.append(True)
            # Updating values to compare based on latest accepted row
            xcenter = row['xcentroid']
            ycenter = row['ycentroid']
            last_index = index
    result = df.loc[filters]
    return result.reset_index(drop=True)

def find_nearest(sources_1, sources_2):
    positions = np.transpose((sources_1['xcentroid'], sources_1['ycentroid']))
    positions_2 = np.transpose((sources_2['xcentroid'], sources_2['ycentroid']))
    mapping_dict = dict()
    for index, value in enumerate(positions):
        difference = np.subtract(positions_2, value)
        squared_difference = np.square(difference)
        min_index = np.sqrt(squared_difference.sum(axis = 1)).argmin()
        if index in mapping_dict:
            old_difference_in_flux = abs(sources_1['flux'][index] - sources_2['flux'][mapping_dict[index]])
            new_difference_in_flux = abs(sources_1['flux'][index] - sources_2['flux'][min_index])
            if old_difference_in_flux > new_difference_in_flux:
                mapping_dict[index] = min_index
        else:
            mapping_dict[index] = min_index
    for key, item in mapping_dict.items():
        x_1, y_1 = sources_1.iloc[key]['xcentroid'], sources_1.iloc[key]['ycentroid']
        x_2, y_2 = sources_2.iloc[item]['xcentroid'], sources_2.iloc[item]['ycentroid']
        distance = np.sqrt((x_1-x_2)**2+(y_1-y_2)**2)
        if distance > 60:
            mapping_dict[key] = np.nan
    return mapping_dict



In [ ]:
dictionary_list = list()
sources_first_array = find_sources(data[0])
trimmed_sources_first_array = image_trimmer(sources_first_array, data[0])
sources_first_array_no_duplicates = excluding_duplicates(trimmed_sources_first_array, 9)

for index in range(1, len(data)):
    sources_next_array = find_sources(data[index])
    trimmed_sources_next_array = image_trimmer(sources_next_array, data[index])
    sources_next_array_no_duplicates = excluding_duplicates(trimmed_sources_next_array, 9)
    mapping_dictionary = find_nearest(sources_first_array_no_duplicates, sources_next_array_no_duplicates)
    dictionary_list.append(mapping_dictionary)

In [ ]:
method_2_dict = dict()
method_2_dict[0] = list(dictionary_list[0].keys())
for index, dictionary in enumerate(dictionary_list):
    method_2_dict[index+1] = list(dictionary.values())

In [ ]:
method_2_list_sources = list()

for key, item in method_2_dict.items():
    temp_list = list()
    found_sources = find_sources(data[key])
    trimmed_image = image_trimmer(found_sources, data[key])
    final_df = excluding_duplicates(trimmed_image, 9)
    for i in item:
        if i is np.nan:
            temp_list.append(np.nan)
        else:
            # temp_list.append(final_df.iloc[i]['flux'])
            temp_list.append((final_df.iloc[i]['xcentroid'], final_df.iloc[i]['ycentroid']))
    method_2_list_sources.append(temp_list)

In [ ]:
all_positions_dict = dict()
for lst in method_2_list_sources:
    for index, item in enumerate(lst):
        if index not in all_positions_dict.keys():
            all_positions_dict[index] = list()
            all_positions_dict[index].append(item)
        else:
            all_positions_dict[index].append(item)

In [ ]:
position_df = pd.DataFrame(all_positions_dict)

In [ ]:
list(position_df.loc[0])

In [ ]:
from photutils.aperture import CircularAperture, CircularAnnulus, ApertureStats, aperture_photometry
from astropy.stats import SigmaClip
import itertools

def signal_to_noise_ratio_v3(image_array, positions_df, r_start, r_end, step_size, delta_r, return_df = False):
    sigclip = SigmaClip(sigma=3, maxiters=5)
    # List of the aperture radius
    aperture_radius = list()

    # List of lists with the signal-to-noise ratios at each aperture
    all_snr_list = list()

    # Empty dataframe for putting everything in later
    snr_df = pd.DataFrame()

    # Get the positions that have a nan in them and ignore them
    nan_indexes = np.argwhere(pd.isnull(positions_df))
    if nan_indexes.size > 0: # Only works with a np.array
        no_nan_positions = list(positions_df[~pd.isnull(positions_df)])
    else: # Only works with a list
        no_nan_positions = list(positions_df)

    for r in np.arange(r_start, r_end + step_size, step_size):

        annulus_aperture = CircularAnnulus(no_nan_positions, r_in = r, r_out = r + delta_r)
        bkg_stats = ApertureStats(image_array, annulus_aperture, sigma_clip=sigclip)
        bkg_median = bkg_stats.median
        apertures = CircularAperture(no_nan_positions, r)
        phot_table = aperture_photometry(image_array, apertures)
        aper_stats = ApertureStats(image_array, apertures, sigma_clip=None)
        aperture_area = aper_stats.sum_aper_area.value
        total_bkg = bkg_median * aperture_area
        phot_bkgsub = phot_table['aperture_sum'] - total_bkg
        signal_to_noise_ratio = list(phot_bkgsub / total_bkg)
        aperture_radius.append(r)
        for nan_index in nan_indexes:
            signal_to_noise_ratio.insert(nan_index[0], np.nan)
        all_snr_list.append(signal_to_noise_ratio)

    # Transpose the snr list
    stars_snr = list(map(list, itertools.zip_longest(*all_snr_list, fillvalue=None)))

    # Add the aperture radii in the first column
    snr_df['Aperture radii'] = aperture_radius

    # Create a column with the snr of each star at different radii
    for star_index, snr_list in enumerate(stars_snr):
        snr_df[f'Star {star_index}'] = snr_list

    indexes_of_max_snr = list(snr_df.idxmax())[1:] # Exclude the first one since it's the aperture radii

    optimal_radii = [snr_df['Aperture radii'].iloc[int(i)] if not np.isnan(i) else np.nan for i in indexes_of_max_snr]
    optimal_radius = np.nanmean(optimal_radii)

    if return_df:
        return snr_df
    else:
        return optimal_radius

In [ ]:
def aperture_calculation(image_array, positions_df, r, delta_r, exp_time, gain, return_table = False):
    sigclip = SigmaClip(sigma=3, maxiters=5)
    magnitudes_list = list()
    fluxes_list = list()
    for position in positions_df:
        if position is not np.nan:
            annulus_aperture = CircularAnnulus(position, r_in = r, r_out = r + delta_r)
            bkg_stats = ApertureStats(image_array, annulus_aperture, sigma_clip=sigclip)
            bkg_median = bkg_stats.median
            apertures = CircularAperture(position, r)
            phot_table = aperture_photometry(image_array, apertures)
            aper_stats = ApertureStats(image_array, apertures, sigma_clip=None)
            aperture_area = aper_stats.sum_aper_area.value
            total_bkg = bkg_median * aperture_area
            phot_bkgsub = phot_table['aperture_sum'] - total_bkg
            phot_table['aperture_sum_bkgsub'] = phot_bkgsub
            flux = (gain * phot_bkgsub) / exp_time
            mag = 25 - 2.5 * np.log10(flux)
            magnitudes_list.append(mag[0])
            fluxes_list.append(flux[0])
        else:
            magnitudes_list.append(np.nan)
            fluxes_list.append(np.nan)
    if return_table:
        return phot_table
    else:
        return magnitudes_list, fluxes_list

In [ ]:
def calculate_magnitude(flux):
    return 25 - 2.5 * np.log10(flux)

In [ ]:
all_magnitudes = list()
all_fluxes = list()
for index in range(len(position_df)):
    print(index)
    positions = np.array(position_df.loc[index])
    array = data[index]
    optimal_aperture_list = signal_to_noise_ratio_v3(array, positions, 1, 30, 0.1, 10)
    magnitudes, fluxes = aperture_calculation(array, positions, optimal_aperture_list, 10, 16, 1)
    all_fluxes.append(fluxes)
    all_magnitudes.append(magnitudes)

In [ ]:
 # Transposing the array of all magnitudes to get the magnitudes of each star in the same list across all arrays

all_magnitudes_transposed = list(map(list, itertools.zip_longest(*all_magnitudes, fillvalue=None)))
all_fluxes_transposed = list(map(list, itertools.zip_longest(*all_fluxes, fillvalue=None)))

## Finding all the stars in the image

To find the stars in the first array we are going to use astrometry to find the RA and Dec of the image, get the corrected header and then use this to pass all the coordinates to sky coordinates. If we do this in the first array, we can then use these coordinates to find out which of the detected stars are non - variable and can be used as standards.

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

# First, find all the sources in the first image

sources_first_image = excluding_duplicates(image_trimmer(find_sources(data[0]), data[0]), 9)

positions_first_image = list(zip(sources_first_image['xcentroid'], sources_first_image['ycentroid']))

for i, j in enumerate(positions_first_image):
    print(i, j)


In [ ]:
sources_first_image

In [ ]:
aperture_calculation(data[0], positions_first_image, 4., 10, 16, 1, return_table= True)

In [ ]:
from convenience_functions import show_image

show_image(data[0])
for position in positions_first_image:
    plt.plot(*position,  marker = 'x', markersize = 5, color = 'green')
plt.plot(358.7714824860396, 1044.1420601306495, marker = 'x', color = 'red', markersize = 5)

In [ ]:
f = fits.open('../reduced-lights_2022_03_20/Luminance/new-image.fits')
w = WCS(f[0].header)

for index, coords in enumerate(positions_first_image):
    sky = w.pixel_to_world(coords[0], coords[1]).data
    print(index, sky)
f.close()

By looking at their positions and crossing that with SIMBAD we can see which of these sources are stars we can use as local standards. For this particular case the list is

`stars_index = [0,2,13,25,33,34]`

and RV UMa is star 34 in the first array

In [ ]:
RV_UMa_index = 19

RV_UMa_magnitudes = [all_magnitudes_transposed[RV_UMa_index]]
RV_UMa_fluxes = [all_fluxes_transposed[RV_UMa_index]]

In [ ]:
saturated_stars_and_RV_UMa = [1,11,19,24,35]
stars_index =[i for i in range(len(sources_first_image)) if i not in saturated_stars_and_RV_UMa]

stars_index

In [ ]:
magnitude_list_standard_stars = [all_magnitudes_transposed[i] for i in stars_index]
flux_list_standard_stars = [all_fluxes_transposed[i] for i in stars_index]

We can plot all these stars to see which of them can be useful, since the nearest neighbours algorithm is not perfect.

In [ ]:
for index, lst in enumerate(magnitude_list_standard_stars):
    plt.plot(lst, 'x')
    plt.title(f'Star {stars_index[index]}')
    plt.xlabel('Array number')
    plt.ylabel('Magnitude')
    plt.gca().invert_yaxis()
    plt.show()
    plt.close()

Looking at these images, we can clearly tell there are some problems with star 19, star 41 and star 50. Star 58 also has one in the middle which is clearly not matching the other ones. These ones may be skewing our correction for RV UMa, so let's remove them.

In [ ]:
# Removing the arrays with several errors
bad_stars_index = [1,6,7,11,12,15,16,19,20,22,24,26,27,29,30,35]
corrected_stars_index = [i for i in range(len(sources_first_image)) if i not in bad_stars_index]

flux_list_standard_stars = [all_fluxes_transposed[i] for i in corrected_stars_index]
magnitude_list_standard_stars = [all_magnitudes_transposed[i] for i in corrected_stars_index]

# Now we remove the item in the last array that is a bit weird and substitute it with a nan

max_index = np.nanargmax(flux_list_standard_stars[1])

flux_list_standard_stars[1][max_index] = np.nan
magnitude_list_standard_stars[1][max_index] = np.nan

In [ ]:
# Check if everything is correct

plt.plot(magnitude_list_standard_stars[1], 'x')
plt.xlabel('Array number')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()

In [ ]:
# Now we transpose the all_magnitudes list so that each list contains all the magnitudes of each star
all_flux_offsets_standard_stars = list()
all_magnitude_offsets_standard_stars = list()
for index, flux_list in enumerate(all_fluxes_transposed):
    if index in corrected_stars_index:
        offset = [flux_list[0]- i for i in flux_list]
        all_flux_offsets_standard_stars.append(offset)

for index, flux_list in enumerate(all_magnitudes_transposed):
    if index in corrected_stars_index:
        offset = [flux_list[0]- i for i in flux_list]
        all_magnitude_offsets_standard_stars.append(offset)

In [ ]:
mean_magnitude_offsets = np.nanmean(all_magnitude_offsets_standard_stars, axis = 0)
mean_flux_offsets = np.nanmean(all_flux_offsets_standard_stars, axis = 0)

std_magnitude_offsets = np.nanstd(all_magnitude_offsets_standard_stars, axis = 0)
std_flux_offsets = np.nanstd(all_flux_offsets_standard_stars, axis = 0)

In [ ]:
plt.plot(std_magnitude_offsets, 'x')

In [ ]:
from operator import add
corrected_RV_UMa_flux = list(map(add, RV_UMa_fluxes, mean_flux_offsets))[0]
corrected_RV_UMa_magnitude = list(map(add, RV_UMa_magnitudes, mean_magnitude_offsets))[0]

In [ ]:
corrected_RV_UMa_magnitude

# Stuff for the presentation

In [ ]:
test_sources_first_array = excluding_duplicates(image_trimmer(find_sources(data[0]), data[0]), 9)
test_sources_last_array = excluding_duplicates(image_trimmer(find_sources(data[-1]), data[-1]), 9)

In [ ]:
test_sources_first_array

In [ ]:
test_sources_last_array

In [ ]:
dict_presentation = find_nearest(test_sources_first_array, test_sources_last_array)

In [ ]:
dict_presentation

In [ ]:
for key in dict_presentation.keys():
    if key in corrected_stars_index:
        print(key, dict_presentation[key])

In [ ]:
last_array_mapping = [2,5,21,39,46,48]

In [ ]:
list(zip(test_sources_first_array.iloc[corrected_stars_index]['xcentroid'], test_sources_first_array.iloc[corrected_stars_index]['ycentroid']))

In [ ]:
list(zip(test_sources_last_array.iloc[last_array_mapping]['xcentroid'], test_sources_last_array.iloc[last_array_mapping]['ycentroid']))

In [ ]:
positions = np.transpose((test_sources_first_array['xcentroid'], test_sources_first_array['ycentroid']))

apertures = CircularAperture(positions, r=4.)

norm = ImageNormalize(stretch=SqrtStretch())

show_image(data[0], cmap = 'gray')

apertures.plot(color='blue', lw=1.5, alpha=0.5)

plt.plot(2308, 1780, marker = 'x', markersize = 10)

plt.axhline(data[0].shape[0]/2)

plt.axvline(data[1].shape[1]/2);

print(data[0].shape[0]/2, data[1].shape[1]/2)

In [ ]:
from PIL import Image, ImageOps
from IPython.display import  display

img = Image.open('20220322_detected_sources.jpg')
img = ImageOps.flip(img)
display(img)

In [ ]:
first_array_sources_positions = [(3156.391593667612, 249.55410761820283),
                                 (4241.96236981832, 402.72851815537894),
                                 (424.6783347656137, 1374.9292357923953),
                                 (345.5587801311043, 2684.932418493146),
                                 (1754.1996705503554, 3141.7386564632407),
                                 (396.773250967703, 3296.865105632183)]

In [ ]:
last_array_sources_positions = [(3198.6312350867915, 275.13056433320367),
                                (4283.939991823673, 429.0982434051284),
                                (466.61276952428346, 1399.5207879354884),
                                (387.837585300374, 2709.4622790306116),
                                (1795.594390834655, 3166.587148487002),
                                (438.69571061757875, 3321.373967753316)]

In [ ]:
corrected_stars_index

In [ ]:
show_image(data[0], cmap= 'gray')
plt.plot(3156.391593667612, 249.55410761820283, marker = 'x', markersize = 7, color = 'red', label = 'TYC 3850-91-1')
plt.plot(4241.96236981832, 402.72851815537894, marker = 'x', markersize = 7, color = 'green', label = 'TYC 3850-738-1')
plt.plot(424.6783347656137, 1374.9292357923953, marker = 'x', markersize = 7, color = 'blue', label = 'TYC 3850-209-1')
plt.plot(345.5587801311043, 2684.932418493146, marker = 'x', markersize = 7, color = 'purple', label = 'TYC 3850-654-1')
plt.plot(1754.1996705503554, 3141.7386564632407, marker = 'x', markersize = 7, color = 'yellow', label = 'TYC 3850-28-1')
plt.plot(396.773250967703, 3296.865105632183, marker = 'x', markersize = 7, color = 'peru', label = 'TYC 3850-753-1')
plt.legend();

In [ ]:
from convenience_functions import show_image
show_image(data[-1], cmap= 'gray')
plt.plot(3198.6312350867915, 275.13056433320367, marker = 'x', markersize = 7, color = 'red', label = 'TYC 3850-91-1')
plt.plot(4283.939991823673, 429.0982434051284, marker = 'x', markersize = 7, color = 'green', label = 'TYC 3850-738-1')
plt.plot(466.61276952428346, 1399.5207879354884, marker = 'x', markersize = 7, color = 'blue', label = 'TYC 3850-209-1')
plt.plot(387.837585300374, 2709.4622790306116, marker = 'x', markersize = 7, color = 'purple', label = 'TYC 3850-654-1')
plt.plot(1795.594390834655, 3166.587148487002, marker = 'x', markersize = 7, color = 'yellow', label = 'TYC 3850-28-1')
plt.plot(438.69571061757875, 3321.373967753316, marker = 'x', markersize = 7, color = 'peru', label = 'TYC 3850-753-1')
plt.legend();

In [ ]:
plt.plot(magnitude_list_standard_stars[0], 'x', label = 'Magnitude')
plt.axhline(np.nanmean(magnitude_list_standard_stars[0]), label ='Mean magnitude')
plt.xlabel('Array number')
plt.ylabel('Magnitude')
plt.legend();

In [ ]:
rows = 4
columns = 3
index = 0

for index, lst in enumerate(magnitude_list_standard_stars):
    plt.plot(lst, 'x')
    plt.title(f'Star {index}')
    plt.show()
    plt.close()

In [ ]:
np.argmax(magnitude_list_standard_stars[9])

In [ ]:
plt.plot(all_magnitude_offsets_standard_stars[0], 'x', color = 'blue', label ='Offset star 0')
plt.ylabel('Offset')
plt.xlabel('Array number')
plt.title('Offset star 0')

In [ ]:
plt.plot(all_magnitude_offsets_standard_stars[3], 'x', color = 'blue', label ='Offset star 0')
plt.ylabel('Offset')
plt.xlabel('Array number')
plt.title('Offset star 3')

In [ ]:
from operator import add
test_lst = list(map(add, flux_list_standard_stars[0], all_magnitude_offsets_standard_stars[0]))
test_lst

In [ ]:
plt.plot(test_lst, 'x')
plt.ylabel('Corrected Flux')
plt.xlabel('Array number')
plt.title('Corrected magnitude star 0')

In [ ]:
plt.plot(corrected_RV_UMa_flux, 'x')
plt.xlabel('Array number')
plt.ylabel('Flux')
plt.title('Corrected RV UMa magnitude')

In [ ]:
for lst in all_magnitude_offsets_standard_stars:
    print(lst[24])

In [ ]:
mean_magnitude_offsets

In [ ]:
find_nearest(test_sources_first_array, excluding_duplicates(image_trimmer(find_sources(data[25]), data[25]), 9))

In [ ]:
excluding_duplicates(image_trimmer(find_sources(data[25]), data[25]), 9)

In [ ]:
show_image(data[25])
plt.plot(2351, 1784, marker = 'x', markersize = 5, color = 'green')
plt.plot(2331, 1738, marker = 'x', markersize = 5, color = 'red')

# Plotting the arrays with time in the x - axis

In [ ]:
import datetime

times = [datetime.datetime.strptime(header["DATE-OBS"], '%Y-%m-%dT%H:%M:%S.%f') for header in headers]
t = [(time - times[0]).total_seconds() for time in times]

In [ ]:
plt.plot(t, corrected_RV_UMa_magnitude, 'x')
plt.xlabel('time (seconds)')
plt.ylabel('Magnitude');

In [ ]:
fwhm_data = pd.read_csv('ds9.dat', sep = ' ', header = None)
plt.plot(fwhm_data[0], fwhm_data[1])
plt.axhline(max(fwhm_data[1])/2, color = 'green')

# TO DO

Once the algorithm has been done for all the arrays, check on a plot how the magnitude changes. The last star detected on the first array with the current parameters is very close to two other stars, so that when the last arrays are being ran its nearest neighbour is actually another star. We do not need this star for the photometry, so just ignore it. The 5th star in the first array is RV UMa in the first day. In the second it's number 3

In [ ]:
# TO DO
# 1. Test flux lists with all 55 arrays
# 2. Plot the fluxes across all the arrays
# 3. Check if any stars are confused with others
# 4. Grab the positions of all the stars in the array and their mappings back to the first array
# 5. Run the same procedure on the other days



In [ ]:
all_sources_all_arrays = list() # list with the sources on all the arrays that correspond to sources in the first image

all_sources_all_arrays.append(list(dictionary_list[0].keys()))
for index in range(len(dictionary_list)):
    sources_n_array = list() # What the sources in array n correspond to in the first array
    for key in dictionary_list[0].keys():
        sources_n_array.append(dictionary_list[index][key])
    all_sources_all_arrays.append(sources_n_array)

In [ ]:
positions_dict = dict()

for index, lst in enumerate(all_sources_all_arrays):
    found_sources = find_sources(data[index])
    trimmed_image = image_trimmer(found_sources, data[index])
    final_df = excluding_duplicates(trimmed_image, 9)
    for position_first_array, position_nth_array in enumerate(lst):
        if position_first_array in positions_dict.keys():
            positions_dict[position_first_array].append(position_nth_array)
        else:
            positions_dict[position_first_array] = list()

In [ ]:
flux_lists = list()
for key, chain in positions_dict.items():
    fluxes = list()
    for i in range(len(chain)):
        if i == 0:
            found_sources = find_sources(data[i])
            trimmed_sources = image_trimmer(found_sources, data[i])
            sources_no_duplicates = excluding_duplicates(trimmed_sources, 9)
            fluxes.append(sources_no_duplicates.iloc[key]['flux'])
        else:
            j = chain[i-1]
            if j is not np.nan:
                found_sources = find_sources(data[i])
                trimmed_sources = image_trimmer(found_sources, data[i])
                sources_no_duplicates = excluding_duplicates(trimmed_sources, 9)
                fluxes.append(sources_no_duplicates.iloc[j]['flux'])
            else:
                fluxes.append(np.nan)
    flux_lists.append(fluxes)